In [1]:
%load_ext autoreload
%autoreload 2


In [9]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical, img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Input, concatenate
from tensorflow.keras.callbacks import EarlyStopping
from ml_logic.preprocessor import data_balancing, create_model, load_and_preprocess_image, image_augmentation, disease_categorization, num_cols, create_model_stage2, disease_cat_eval
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input


In [3]:
# Loading Datasets

train = pd.read_csv('../data/RFMiD_Training_Labels.csv').set_index('ID')
test = pd.read_csv('../data/RFMiD_Testing_Labels.csv').set_index('ID')
eval = pd.read_csv('../data/RFMiD_Validation_Labels.csv').set_index('ID')


In [4]:
# Splitting Data

X_train = disease_categorization(table_link='../data/RFMiD_Training_Labels.csv')
y_train = X_train
X_eval  = disease_cat_eval(table_link='../data/RFMiD_Validation_Labels.csv', disease_cat_df=X_train)
y_eval = X_eval


In [24]:
def load_image(image_id, image_folder, target_size=(224, 224)):
    """
    Load and preprocess an image given its ID.
    """
    image_path = os.path.join(image_folder, f'{image_id}.png')
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    return image


In [25]:
# Loading Images

image_folder = '../data/training_images'
eval_image_folder = '../data/eval_images'
# images = np.array([load_and_preprocess_image(row_id, image_folder) for row_id in X_train.index])
# eval_images = np.array([load_and_preprocess_image(row_id, eval_image_folder) for row_id in X_eval.index])
images = np.array([load_image(row_id,image_folder) for row_id in X_train.index])
eval_images = np.array([load_image(row_id,eval_image_folder) for row_id in X_eval.index])


KeyboardInterrupt: 

In [10]:
# Augment Images

augmented_images = image_augmentation(images)
augmented_eval_images = image_augmentation(eval_images)
print(augmented_images.shape)
print(augmented_eval_images.shape)

plt.imshow(augmented_images[0])
plt.imshow(augmented_eval_images[0])


2023-12-05 19:04:41.896235: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-12-05 19:04:41.898421: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-12-05 19:04:41.900690: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-R72GEULP): /proc/driver/nvidia/version does not exist
2023-12-05 19:04:41.913132: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 19:04:42.084477: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 638840832 e

2023-12-05 19:04:56.584696: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 638840832 exceeds 10% of free system memory.


2023-12-05 19:05:16.829749: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 638840832 exceeds 10% of free system memory.


: 

when comiling, it's categorical cross entropy instead of binary cross entropy

In [7]:
base_model = VGG16(weights = "imagenet", include_top = False, input_shape = (224, 224, 3))


2023-12-09 15:20:39.924891: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-12-09 15:20:39.926361: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-12-09 15:20:39.927930: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-R72GEULP): /proc/driver/nvidia/version does not exist
2023-12-09 15:20:39.944245: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


58889256/58889256 [==============================] - 8s 0us/step


In [17]:
# NEW MODEL

image_input = tf.keras.layers.Input(shape=(224, 224, 3))
#x = data_augmentation(image_input) # We still have our data augmentation layers
x = preprocess_input(image_input) # Then a preprocessing layer specifically designed for the VGG16
x = base_model(x) # Then our transfer learning model
# x = Conv2D(64, (3, 3), activation=‘relu’, padding = “same”)(x)
# x = MaxPooling2D((2, 2),padding = “same”)(x)
# x = Conv2D(32, (3, 3), activation=‘relu’, padding = “same”)(x)
# x = MaxPooling2D((2, 2),padding = “same”)(x)
x = Conv2D(1024, (3, 3), activation='relu', padding = "same")(x)
x = MaxPooling2D((2, 2),padding = "same")(x)
x = Flatten()(x)
z = Dense(64, activation='relu')(x)
#z = layers.Dropout(0.3)(z)
z = Dense(32, activation='relu')(z)
#z = layers.Dropout(0.3)(z)
z = Dense(5, activation='softmax')(z)
model_test1 = Model(inputs=image_input, outputs=z)
# And we freeze the VGG16 model
base_model.trainable = False


In [18]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)


In [19]:
model_test1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['Precision', 'accuracy'])


In [23]:
history3 = model_test1.fit(
    augmented_images, y_train,
    validation_data=(augmented_eval_images, y_eval),
    epochs=50,
    batch_size=32,
    callbacks=[es2, LRreducer]
)


NameError: name 'augmented_images' is not defined